# Spaceship Titanic from Kaggle

#### Dataset description (from Kaggle)


In this competition your task is to predict whether a passenger was transported to an alternate dimension during the Spaceship Titanic's collision with the spacetime anomaly. To help you make these predictions, you're given a set of personal records recovered from the ship's damaged computer system.



#### File and Data Field Descriptions

-  __train.csv__ - Personal records for about two-thirds (approx 8700) of the passengers, to be used as training data.


    - _PassengerId_ - A unique Id for each passenger. Each Id takes the form _gggg_pp_ where _gggg_ indicates a group the passenger is travelling with and _pp_ is their number within the group. People in a group are often family members, but not always.

    - _HomePlanet_ - The planet the passenger departed from, typically their planet of permanent residence.

    - _CryoSleep_ - Indicates whether the passenger elected to be put into suspended animation for the duration of the voyage. Passengers in cryosleep are confined to their cabins.

    - _Cabin_ - The cabin number where the passenger is staying. Takes the form _deck/num/side_, where side can be either _P_ for Port or _S_ for Starboard.

    - _Destination_ - The planet the passenger will be debarking to.

    - _Age_ - The age of the passenger.

    - _VIP_ - Whether the passenger has paid for special VIP service during the voyage.

    - _RoomService_, _FoodCourt_, _ShoppingMall_, _Spa_, _VRDeck_ - Amount the passenger has billed at each of the Spaceship Titanic's many luxury amenities.

    - _Name_ - The first and last names of the passenger.

    - _Transported_ - Whether the passenger was transported to another dimension. This is the target, the column you are trying to predict.


- __test.csv__ - Personal records for the remaining one-third (approx 4300) of the passengers, to be used as test data. Your task is to predict the value of _Transported_ for the passengers in this set.

- __sample_submission.csv__ - A submission file in the correct format.

    - _PassengerId_ - Id for each passenger in the test set.


    - _Transported_ - The target. For each passenger, predict either _True_ or _False_.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
df = pd.read_csv("../Data/train.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB
